In [1]:
import pandas as pd
import datetime
import requests
import re
from bs4 import BeautifulSoup as bs
from dateutil.relativedelta import *
from sqlalchemy import create_engine
import os
from app import db
from models import Historical, Forecast, Station, Current

In [9]:
which python

SyntaxError: invalid syntax (<ipython-input-9-d8603f9f9bc3>, line 1)

In [ ]:
df = get_forecasts()
df.head()

In [2]:
def get_forecasts():
    # 5-day 3-h forecast (only free one available)
    url = 'http://api.openweathermap.org/data/2.5/forecast?id=6167865&APPID=631d59f50ab1841ba7af0f0f706e1505'
    response = requests.get(url, verify=True)
    r = response.json()
    
    if response:
        df=pd.DataFrame()
        for dt in r['list']:    
            row={'datetime':dt['dt_txt'],
                 'retrieval_time':datetime.datetime.now(),
                 'drybulb':dt['main']['temp']-273.15,
                 'relative_humidity':dt['main']['humidity']}
            # note: Kelvin to Celcius
            
            df = df.append(row, ignore_index=True)
        df.set_index(pd.DatetimeIndex(df['datetime']), inplace=True)
        return df[['retrieval_time','drybulb', 'relative_humidity']]        

    else:
        print('Response Error')

In [3]:
def add_forecasts():
    session = db.session     # Creates a SQLAlchemy Session
        
    try:
        df = get_forecasts()
        for row in df.itertuples(): 
            data = Forecast(id=row[0], 
                            retrieval_time=row[1],
                            drybulb=row[2], 
                            relative_humidity=row[3])
            session.merge(data)
            session.commit()

    except:
        print('something messed up') 

In [ ]:
# df.to_csv('~/Desktop/test.csv')

In [ ]:
# Legacy
def add_scraped_forecasts():
    import time
    session = db.session     # Creates a SQLAlchemy Session
    tries = 0
   
    while tries <=10:
        try:
            df = scrape_forecasts()
            for row in df.itertuples(): 
                data = Forecast(id=row[0], drybulb=row[1], dewpoint=row[2])
                session.merge(data)
                session.commit()
            break

        except:
            print('can\'t get forecast')
            time.sleep(60)
            tries +=1